# Bragg beam splitter

## Physics

The Bragg beam splitter splits the wave function into two momentum states (but same internal state). 
Here, the main physics is sketched. 
The interested reader is referred to [the textbook by Grynberg, Aspect and Fabre](https://www.cambridge.org/core/books/introduction-to-quantum-optics/F45DCE785DC8226D4156EC15CAD5FA9A). 

The formalism describing the Bragg atom-light interaction is based on a semi-classical model. 
The atoms are described quantum mechanically via a wave function, while the light is described classically (its high intensity ensures a substantial photon presence at all times).
The Hamiltonian of the system is:
$$
    \hat H = \frac{\hat p^2}{2m} - \hat{\vec D} \vec E_L (\vec r, t)
$$
where $\hat p$ is the momentum operator, $\hat{\vec D}$ is the dipole operator and $\vec E_L$ is the electric field. 

The electric field can be described by two counterpropagating laser beams with frequencies $\omega_L+\omega_r$ (drives the transition $| g,0 \rangle \rightarrow | e, \hbar k_L \rangle$) and $\omega_r$ (drives the transition $| e, \hbar k_L \rangle \rightarrow | g, 2\hbar k_L\rangle$). 
Note that $\frac{(2\hbar k_L)^2}{2m} = \hbar \omega_r$.
It should be noted that both lasers are detuned by $\Delta$, such that the transition $|g,0 \rangle \rightarrow | e, \hbar k_L \rangle$ is unlikely to happen without the stimulated emission directly after it. 

Here, a one dimensional wave function $\Psi (x)$ is considered. 
Adiabatic elimination of the excited state leads to
$$
\hat H = -\frac{\hbar^2}{2m}\nabla^2 + 2 \hbar \Omega \cos ^2 \left( k_L x - \frac{\omega_r}{2} t \right)
$$
where $\Omega$ is the effective Rabi frequency. 
$\Omega$ is determined by the laser properties and has typically a Gaussian temporal profile to ensure good velocity selectivity. 
If the atoms are freely falling, an acceleration term $\frac{1}{2}a_\text{laser}t^2$ is added to the laser phase to ensure that the laser beams stay resonant to the falling atoms. 
Also common is an additional constant phase shift $\Phi_0$.

After the atom-light interaction, the atom is left in a superposition of states $|g,0\rangle$ and $|g,2\hbar k_L\rangle$, and typically higher orders like $|g,-2\hbar k_L\rangle$ and $|g,4\hbar k_L\rangle$ [[Siemß 2020](https://link.aps.org/doi/10.1103/PhysRevA.102.033709)]. 
Idealized, this sequence applies a momentum transfer of $2\hbar k_L$ to the atom with a $50\%$ chance.


## Implementation
This example illustrates the implementation of propagating a wave function through a Bragg beam splitter.
We propagate a wave function (described by an `fa.Array`) under the influence of the time-dependent Bragg beam splitter potential.

In [ ]:
from typing import Tuple, Dict, Any
import numpy as np
import jax
import jax.numpy as jnp
from bokeh.io import output_notebook

import fftarray as fa
from helpers import plt_array, plt_array_values_space_time

# setup bokeh in jupyter notebook
output_notebook()

# setup jax
jax.config.update("jax_enable_x64", True)
fa.set_default_xp(jnp)
fa.jax_register_pytree_nodes()

### Initialize global constants

In [2]:
from scipy.constants import hbar, pi, physical_constants
# angular frequency used to initialize the ground state (of quantum harmonic
# oscillator)
omega_x = 2*pi*10 # Hz
# laser pulse parameters
# Rabi frequency. This specific value was found as a binary search to
# optimize a 50/50 split of the two momentum classes for this specific beam
# splitter duration and pulse form.
rabi_frequency = 25144.285917282104 # Hz
phi_0 = 0. # phase jump
bragg_acc = 0. # bragg acceleration
sigma_bs = 25e-6 # temporal pulse width (s)

# Rb87 properties
m = 86.909 * physical_constants["atomic mass constant"][0] # mass in kg
lambda_L = 780 * 1e-9 # wavelength in m
k_L = 2 * pi / lambda_L # wave number in 1/m
hbark: float = hbar * k_L # momentum transfer
hbarkv: float = hbark/m # recoil velocity
w_r = 2 * hbarkv * k_L # recoil frequency in rad/s

#### Define the time grid
In order to simulate the time evolution of the wave function, we need to discretize 
time into intervals of length $\Delta t$ in between the potential is
assumed constant. Then, for each time step, we can apply the time evolution 
operator as 
$|\Psi(t+\Delta t)\rangle = e^{-i \hat H \Delta t /\hbar}|\Psi(t)\rangle$ 
using the split step method.

Here, we choose to sample $4\sigma_\mathrm{bs}$ of the Gaussian temporal profile
of the Bragg beam splitter potential with Gaussian width $\sigma_\mathrm{bs}$ 
with a step size of $\Delta t = 1$ µs. Additionally, we let the wave function 
freely propagate for $25$ ms with $\Delta t = 50$ µs after applying the beam 
potential to illustrate the separation of both momentum states in position 
space.

```text
 laser start      intensity peak        laser end            simulation end
------|-----------------|-------------------|----------------------|-----> time
      |------------ 4 sigma_bs -------------|-- free propagation --|
```

In [3]:
# define how many sigmas of the gauss to sample in each direction before
# reaching zero intensity:
sampling_range_mult = 4. # * sigma_bs
dt_bs = 1e-6 # time step size
# total number of pulse grid steps = gauss_width * scaling_factor / step_size
steps_limit = int(round(sigma_bs * sampling_range_mult / dt_bs))
t_offset = steps_limit*dt_bs
nt_bs = 2*steps_limit # number of time steps for beam splitter
dt_free = 5e-5 # defines time step size for free propagation
nt_free = 50 # number of time steps for free propagation
# time lists
t_list_bs: Any = np.arange(nt_bs)*dt_bs
t_list_free = t_list_bs[-1]+np.arange(1,nt_free+1)*dt_free
t_list = np.concatenate((t_list_bs, t_list_free))

### Initialize the wavefunction
We initialize the wavefunction as the groundstate of a quantum harmonic oscillator with frequency $\omega_\mathrm{QHO} = 2\pi\times 10$ Hz.

In [ ]:
# coordinate grid
x_dim: fa.Dimension = fa.dim_from_constraints(
    name = "x",
    pos_min = -50e-6,
    pos_max = 50e-6,
    freq_middle = 0.,
    freq_extent = 32*k_L,
    loose_params = ["freq_extent"],
    dynamically_traced_coords=False
)

# initialize an Array as harmonic oscillator groundstate
x_coords: fa.Array = fa.coords_from_dim(x_dim, "pos")
psi_init: fa.Array = (m * omega_x / (pi*hbar))**(1./4.) * fa.exp(-(m * omega_x * (x_coords**2.)/(2.*hbar)))
psi_init = psi_init / fa.integrate(fa.abs(psi_init)**2)
plt_array(psi_init)

### Define the potential
Now, we implement the external potential
$$
    V = 2 \hbar \Omega(t) \cos ^2 \left( k_L x - \frac{\omega_r}{2} t \right)
$$
where the time dependent Rabi frequency is defined by
$$
    \Omega(t) = \Omega_0 \exp(-t^2/(2\sigma_\mathrm{bs}^2)).
$$

In [5]:
from scipy.constants import hbar

def V(ramp: float, t: float) -> fa.Array:
    """Bragg pulse potential.

    Parameters
    ----------
    ramp : float
        The pulse ramp (scaling the rabi frequency).
    t : float
        The global lab time.

    Returns
    -------
    fa.Array
        The potential at time t.
    """
    return rabi_frequency * ramp * 2. * hbar * fa.cos(
        k_L * (x_coords - 0.5 * bragg_acc * t**2)
        - 0.5 * w_r * t
        + phi_0/2.
    )**2

Now, we plot the potential at peak intensity (`t=t_offset`).

In [ ]:
from bokeh.plotting import figure, show
# plot the energy trend during the imaginary time evolution
plt = figure(
    width=800, height=400, min_border=50,
    title="The potential versus the wavefunction's position grid",
    x_axis_label="Iteration step",
    y_axis_label="Potential/hbar [Hz]"
)
plot = plt.line(
    x_dim.np_array(space="pos"), V(ramp=1, t=t_offset).values("pos")/hbar
)
show(plt)

### Propagate the wavefunction
Now, we implement the iterative procedure of evolving the wavefunction in time according to the previously defined potential and the subsequent free propagation. 
For this, we need to define the step function that we will iterate via the `jax.lax.scan` function.

#### Define the step functions

<div class="alert alert-block alert-info">
<b>Note:</b> We need to import jax.numpy to calculate the temporal ramp of the 
Bragg beam potential in order to use it in jax.lax.scan. The potential is 
defined using numpy as this is internally handled via jax.numpy when the 
JaxTensorLib was specified in the initialization of the fa.Array.
</div>

In [ ]:
# helper function for the temporal ramp of the beam's intensity
def gauss(t: float, sigma: float):
    """Evaluate a Gaussian function.

	Parameters
	----------
	t : float
		Time value.
	sigma : float
		Width.

	Returns
	-------
	float
		The Gaussian function at t with width sigma.
    """
    return jnp.exp(-0.5 * (t / sigma)**2)

# compute the boundary value of the gauss to ensure that the potential is zero
# when we start and stop applying of the potential
gauss_offset = gauss(t = -t_offset, sigma = sigma_bs)

# momentum operator squared
k_sq = (2*pi*fa.coords_from_arr(psi_init, x_dim.name, "freq"))**2.

def step_bs(psi: fa.Array, t: float) -> Tuple[fa.Array, Dict[str, jax.Array]]:
    """Step function for jax.lax.scan. Apply the Bragg beam splitter pulse.

	Parameters
	----------
	psi : fa.Array
		The wavefunction.
	t : float
		The time.

	Returns
	-------
	fa.Array
		The final wavefunction.
    """
    ramp = gauss(t=t-t_offset, sigma=sigma_bs) - gauss_offset
    # Apply half kinetic propagator
    psi = psi.into_space("freq") * fa.exp((-1.j * 0.5*dt_bs * hbar / (2*m)) * k_sq)

    # Apply potential propagator
    psi = psi.into_space("pos") * fa.exp((-1.j / hbar * dt_bs) * V(ramp, t))

    # Apply half kinetic propagator
    psi = psi.into_space("freq") * fa.exp((-1.j * 0.5*dt_bs * hbar / (2*m)) * k_sq)
    return psi, {
        "abs_pos": fa.abs(psi.into_space("pos")).values("pos")**2,
        "abs_freq": fa.abs(psi.into_space("pos")).values("freq")**2,
    }

def step_free(psi: fa.Array, t: float) -> Tuple[fa.Array, Dict[str, jax.Array]]:
    """Step function for jax.lax.scan. Freely propagate the wavefunction.

	Parameters
	----------
	psi : fa.Array
		The wavefunction.
	t : float
		The time.

	Returns
	-------
	fa.Array
		The final wavefunction.
    """
    psi = psi.into_space("freq") * fa.exp((-1.j * dt_free * hbar / (2*m)) * k_sq)
    return psi, {
        "abs_pos": fa.abs(psi.into_space("pos")).values("pos")**2,
        "abs_freq": fa.abs(psi.into_space("freq")).values("freq")**2,
    }


#### Iterating the step functions

<div class="alert alert-block alert-warning">
<b>Important:</b> The input and output fa.Array of the step function for 
jax.lax.scan has to have the same attributes. This implies that the input and 
output need to be in the same space. Since split_step returns the fa.Array in 
frequency space, we also need to transform the input fa.Array into the 
frequency space before passing it to jax.lax.scan.
</div>

In [8]:
from jax.lax import scan

# bragg beam splitter pulse
psi_final_bs, psi_data_bs = scan(
    f = step_bs,
    init = psi_init.into_space("freq").into_factors_applied(False), # initial wavefunction in frequency space
    xs = t_list_bs
)
# free propagation
psi_final_free, psi_data_free = scan(
    f = step_free,
    init = psi_final_bs.into_space("freq"), # already in frequency space
    xs = t_list_free
)

### Plot the final wavefunction

In [ ]:
# plot the final wave function
plt_array(psi_final_free)

## Plot the complete time evolution

In [ ]:
if nt_bs % nt_free > 0:
    print("Warning: Time grid not equidistant in contour plot.")
psi_init_abs_pos = fa.abs(psi_init.into_space("pos")).values("pos")**2
psi_init_abs_freq = fa.abs(psi_init.into_space("freq")).values("freq")**2
# list of all position space values
plt_data_psi_final_pos = np.concatenate((
    # initial value
    [psi_init_abs_pos],
    # beam splitter values: take only every anim_time_step_mult'th value
    psi_data_bs["abs_pos"][::nt_bs//nt_free],
    # free propagation values
    psi_data_free["abs_pos"]
))
# list of all momentum space values
plt_data_psi_final_freq = np.concatenate((
    # initial value
    [psi_init_abs_freq],
    # beam splitter values: take only every anim_time_step_mult'th value
    psi_data_bs["abs_freq"][::nt_bs//nt_free],
    # free propagation values
    psi_data_free["abs_freq"]
))
plt_data_t_list = np.concatenate((
    [0],
    t_list_bs + dt_bs,
    t_list_free + dt_bs
))

plt_array_values_space_time(
    np.real(plt_data_psi_final_pos),
    np.real(plt_data_psi_final_freq),
    x_dim.np_array("pos"),
    x_dim.np_array("freq")*lambda_L,
    plt_data_t_list,
    freq_unit = "hbar k_L"
)